In [1]:
import sys
print(sys.executable)

import sklearn
sklearn.__version__

/Users/AS/anaconda3/envs/atcs-nli-representations/bin/python


'0.24.1'

In [2]:
import sys
print(sys.executable)

import sklearn
sklearn.__version__

import torch.nn as nn
import torch.optim as optim

import nltk
from nltk import word_tokenize
nltk.download('punkt')

import dill

print('next')
from torchtext import data
from torchtext import datasets
from torchtext.vocab import GloVe
import torch
import torchtext.vocab as vocab
import time


from torchtext.data import Field 
from torchtext.datasets import IMDB
from torchtext.data import BucketIterator
from pytorch_lightning.core.lightning import LightningModule
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger
from torch.optim.lr_scheduler import ReduceLROnPlateau



device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
import tensorboard
from pytorch_lightning.metrics import functional as FM
from pytorch_lightning.callbacks import LearningRateMonitor
from pytorch_lightning.callbacks import EarlyStopping
import pytorch_lightning as pl
from pytorch_lightning import seed_everything
seed_everything(42)

from torch.nn.utils.rnn import pack_padded_sequence,pad_packed_sequence

from utils_eval import *

print(device)


from collections import Counter

/Users/AS/anaconda3/envs/atcs-nli-representations/bin/python


[nltk_data] Downloading package punkt to /Users/AS/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


next


Global seed set to 42
[nltk_data] Downloading package punkt to /Users/AS/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Global seed set to 42


/Users/AS/anaconda3/envs/atcs-nli-representations/bin/python
next
cpu
cpu


In [3]:
class SNLI():
    def __init__(self, args):
        start = time.time()
        #TODO: turn on tokenizer
        self.TEXT = data.Field(batch_first=True, include_lengths=True, tokenize=word_tokenize, lower=True)
#         self.TEXT = data.Field(batch_first=True, include_lengths=True, lower=True)
        self.LABEL = data.Field(sequential=False, unk_token=None)

        # dataset

        self.train, self.dev, self.test = datasets.SNLI.splits(text_field=self.TEXT,
                                                               label_field=self.LABEL,
                                                               root='dataset'
                                                               )
        # train='dataset/snli/snli_1.0_train.jsonl',
        # validation='dataset/snli/snli_1.0_dev.jsonl',
        # test='dataset/snli/snli_1.0_test.jsonl',
        # )
        #
        print('split complete')
        end = time.time()
        print('time for split', end - start)
        print(self.train[0].__dict__.keys())
        print(self.train[0].__dict__.values())

        custom_embeddings = vocab.Vectors(name='../SentEval/pretrained/glove.840B.300d.txt')

        # self.TEXT.build_vocab(self.train, self.dev, self.test, vectors = GloVe(name='840B', dim=300))
        self.TEXT.build_vocab(self.train, self.dev, self.test,
                              vectors=custom_embeddings,  max_size = 37179  )
#         self.LABEL.build_vocab(self.train, max_num = 37179)
        
        
        self.LABEL.build_vocab(self.train, max_size = 37179)
        print('build vocab complete')

        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

        self.train_iter, self.dev_iter, self.test_iter = \
            data.BucketIterator.splits((self.train, self.dev, self.test),
                                       # TODO: change to args
                                       batch_size=64,
                                       device=device)
                                        # batch_size=args.batch_size,
                                        # device=args.gpu)

    def get_iterators(self):
        return self.train_iter, self.dev_iter, self.test_iter, self.TEXT, self.LABEL
    


In [4]:
args = []
s = SNLI(args)
train_iter, dev_iter, test_iter, text, label = s.get_iterators()

/Users/AS/anaconda3/envs/atcs-nli-representations/lib/python3.9/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/Users/AS/anaconda3/envs/atcs-nli-representations/lib/python3.9/site-packages/torchtext/data/example.py:13: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)


split complete
time for split 192.98197197914124
dict_keys(['premise', 'hypothesis', 'label'])
dict_values([['a', 'person', 'on', 'a', 'horse', 'jumps', 'over', 'a', 'broken', 'down', 'airplane', '.'], ['a', 'person', 'is', 'training', 'his', 'horse', 'for', 'a', 'competition', '.'], 'neutral'])
build vocab complete


/Users/AS/anaconda3/envs/atcs-nli-representations/lib/python3.9/site-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [14]:
# class RNN_LSTM(nn.Module):
#     def __init__(self, input_size, embed_size, hidden_size, num_layers):
#         super(RNN_LSTM, self).__init__()
#         self.hidden_size = 2048
#         self.num_layers = 1
#         embed_size = 300

#         self.embedding = nn.Embedding(input_size, embed_size, padding_idx=1)
#         self.rnn = nn.LSTM(input_size = embed_size, hidden_size = self.hidden_size,
#                            num_layers = self.num_layers, bidirectional=False)

#     def forward(self, x):
        
#         src, src_len = x[0], x[1]
#         print(x[0].shape[0])
#         src = self.embedding(src)
        
#         src = pack_padded_sequence(src,src_len,batch_first=True, enforce_sorted=False)
        
#         # Set initial hidden and cell states
#         h0 = torch.zeros(self.num_layers, x[0].shape[0], self.hidden_size)
#         c0 = torch.zeros(self.num_layers, x[0].shape[0], self.hidden_size)
#         output, (hn, cn) = self.rnn(src, (h0, c0))

#         return hn[0]

In [8]:
# class RNN_BiLSTM(nn.Module):
#     def __init__(self, input_size, hidden_size, num_layers, num_classes):
#         super(RNN_BiLSTM, self).__init__()
#         self.hidden_size = 2048
#         self.num_layers = 1
#         embed_size = 300
        
#         self.embedding = nn.Embedding(input_size, embed_size, padding_idx=1)
#         self.rnn = nn.LSTM(input_size = embed_size, 
#                            hidden_size = self.hidden_size,
#                            num_layers = self.num_layers, 
#                            bidirectional=True)
    
#     def forward(self, x):
#         # Set initial states
        
#         src, src_len = x[0], x[1]
#         src = self.embedding(src)
        
#         src = pack_padded_sequence(src,src_len,batch_first=True, enforce_sorted=False)
        
#         h0 = torch.zeros(self.num_layers * 2, x[0].shape[0], self.hidden_size)
#         c0 = torch.zeros(self.num_layers * 2, x[0].shape[0], self.hidden_size)
        
#         # Forward propagate LSTM
#         output, (hn, cn) = self.rnn(src, (h0, c0))
            
#         forward_state=hn[-2,:,:]
#         backward_state=hn[-1,:,:]
      
#         representation=torch.cat((forward_state,backward_state),dim=1)
        
#         # Decode the hidden state of the last time step
#         return representation

In [20]:
# class NLINet(nn.Module):
#     def __init__(self, config, text):
#         super(NLINet, self).__init__()

#         # classifier
#         self.nonlinear_fc = True
#         self.n_classes = 3
#         self.enc_lstm_dim = 300
#         self.dpout_fc = 0.1
#         self.fc_dim = 512
#         self.n_classes = 3

# #         if config.model_name =='BiLSTM':
# #             self.encoder = RNN_BiLSTM(37179, 300, 2048, 1)
# #             self.inputdim = 4 * 4096
# #         if config.model_name =='LSTM':
#         self.encoder = RNN_LSTM(37179, 300, 2048, 1)
#         self.inputdim = 4 * 2048
            
#         #embedding trick        
#         pretrained_embeddings = torch.cat((text.vocab.vectors, torch.zeros(4,300)),0)
#         self.encoder.embedding.weight.data.copy_(pretrained_embeddings)
        
        
#         if self.nonlinear_fc:
#             self.classifier = nn.Sequential(
#                 nn.Dropout(p=self.dpout_fc),
#                 nn.Linear(self.inputdim, self.fc_dim),
#                 nn.Tanh(),
#                 nn.Dropout(p=self.dpout_fc),
#                 nn.Linear(self.fc_dim, self.fc_dim),
#                 nn.Tanh(),
#                 nn.Dropout(p=self.dpout_fc),
#                 nn.Linear(self.fc_dim, self.n_classes),
#                 )
#         else:
#             self.classifier = nn.Sequential(
#                 nn.Linear(self.inputdim, self.fc_dim),
#                 nn.Linear(self.fc_dim, self.fc_dim),
#                 nn.Linear(self.fc_dim, self.n_classes)
#                 )

#     def forward(self, s1, s2):
#         # s1 : (s1, s1_len)
#         u = self.encoder(s1)
#         v = self.encoder(s2)

#         features = torch.cat((u, v, torch.abs(u-v), u*v), 1)
#         output = self.classifier(features)
    
#         return output

#     def encode(self, s1):
#         emb = self.encoder(s1)
#         return emb

# Evaluation + SentEval

In [21]:
# class MyModel(LightningModule):
#     def __init__(self, config, text,train_iter, dev_iter, test_iter):
#         super().__init__()
# #         self.save_hyperparameters()
        
#         self.nli_net = NLINet(config, text)
        
#         weight = torch.FloatTensor(3).fill_(1)
#         self.loss_function = nn.CrossEntropyLoss(weight = weight)
#         self.valid_losses = []
    
#     def forward(self, x):
#         # X is vector of shape (batch, input, )
#         # need to be permuted because by default X is batch first
#         return self.nli_net(x[0], x[1])
    
# #     def on_epoch_end(self):
# #         print('HOOK')
# #         dic = trainer.optimizers[0].param_groups[0]
# #         lr = dic['lr']
# #         print(lr)
        
# #         self.log('learning_rate', lr)
        
# #         #early stopping
# #         if lr<1e-5:
# #             raise KeyboardInterrupt
        
#     def training_step(self, batch, batch_idx):
        

#         premise = batch.premise
#         hypothesis = batch.hypothesis
#         targets = batch.label
#         #TODO: Accuracies in all of the,loops
        
#         y_hat = self.nli_net(premise, hypothesis)
        
#         loss = self.loss_function(y_hat,  targets.type_as(y_hat).long())

#         predict = torch.argmax(y_hat, axis = 1)
#         acc = FM.accuracy(predict, targets)
        
#         metrics_train = {'train_acc': acc, 'train_loss': loss}
#         self.log('train_loss', loss, on_step=True)

#         self.log('train_acc', acc, on_step=True, on_epoch=True, prog_bar=True, logger=True)
#         self.log('train_loss_epoch', loss, on_epoch=True, prog_bar=True, logger=True)
        
#         return loss
        
#     def validation_step(self, batch, batch_idx):
        
        
#         premise = batch.premise
#         hypothesis = batch.hypothesis
#         targets = batch.label
        
#         y_hat = self.nli_net(premise, hypothesis)
#         loss = self.loss_function(y_hat, targets.type_as(y_hat).long())
        
#         predict = torch.argmax(y_hat, axis = 1)
#         acc = FM.accuracy(predict, targets)
#         metrics_val = {'val_acc': acc, 'val_loss': loss}
        
        
#         self.log_dict(metrics_val)
#         self.log('val_acc_on_EPOCH', acc, on_epoch=True)
    
#         return metrics_val
        
#     def test_step(self, batch, batch_idx):
        
        
#         premise = batch.premise
#         hypothesis = batch.hypothesis
#         targets = batch.label
    
        
#         y_hat = self.nli_net(premise, hypothesis)
#         loss = self.loss_function(y_hat, targets.type_as(y_hat).long())
#         predict = torch.argmax(y_hat, axis = 1)

#         predict = torch.argmax(y_hat, axis = 1)
#         acc = FM.accuracy(predict, targets)
        
       
#         metrics_test = {'test_acc': acc, 'test_loss': loss}
#         self.log_dict(metrics_test, prog_bar=True, logger=True)
        

#     def configure_optimizers(self):
#         #TODO: Learning_RATE
#         optimizer = optim.SGD(self.parameters(), lr = 0.1)
# #        
#         scheduler1 = ReduceLROnPlateau(optimizer, mode = 'max', factor = 0.2, 
#                                        verbose = True, patience = 1)

#         return ({'optimizer': optimizer, "lr_scheduler": scheduler1, "monitor": "val_acc"})
    
#     def train_dataloader(self):
#         return train_iter
    
#     def val_dataloader(self):
#         return dev_iter

#     def test_dataloader(self):
#         return test_iter

In [11]:
config = []
cp_path = 'lisalogs/BiLSTM/checkpoints/epoch=8-step=77255.ckpt'

# # 'config', 'text', 'train_iter', 'dev_iter', and 'test_iter'

logger = TensorBoardLogger('Successful_eval', name='BiLSTM')
trainer = Trainer(
        logger=logger,
    )
model_test = MyModel.load_from_checkpoint(
                                         cp_path,config = config, 
                                         text = text, 
                                         train_iter = train_iter, 
                                         dev_iter   = dev_iter,
                                         test_iter  = test_iter)
print(model_test.nli_net.encoder.embedding)
# trainer.test(model_test)




GPU available: False, used: False
TPU available: False, using: 0 TPU cores


RuntimeError: Error(s) in loading state_dict for MyModel:
	Unexpected key(s) in state_dict: "nli_net.encoder.rnn.weight_ih_l0_reverse", "nli_net.encoder.rnn.weight_hh_l0_reverse", "nli_net.encoder.rnn.bias_ih_l0_reverse", "nli_net.encoder.rnn.bias_hh_l0_reverse". 
	size mismatch for nli_net.classifier.1.weight: copying a param with shape torch.Size([512, 16384]) from checkpoint, the shape in current model is torch.Size([512, 8192]).

In [12]:
model_name =  'LSTM'
config = []
cp_path = 'lisalogs/LSTM/checkpoints/epoch=8-step=72963.ckpt'

logger = TensorBoardLogger('Successful_eval', name= model_name)
trainer = Trainer(
        logger=logger,
    )

model_test = MyModel.load_from_checkpoint(
                                         cp_path,
                                         config = config, 
                                         text = text, 
                                         train_iter = train_iter, 
                                         dev_iter   = dev_iter,
                                         test_iter  = test_iter)

print(model_test.nli_net.encoder.embedding)
trainer.test(model_test)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores


Embedding(37179, 300, padding_idx=1)
Testing: 0it [00:00, ?it/s]64
64
Testing:   1%|          | 1/154 [00:00<01:41,  1.51it/s]64
64
Testing:   1%|▏         | 2/154 [00:01<01:40,  1.52it/s]64
64
Testing:   2%|▏         | 3/154 [00:02<02:17,  1.10it/s]64
64
Testing:   3%|▎         | 4/154 [00:03<02:05,  1.20it/s]64
64
Testing:   3%|▎         | 5/154 [00:04<02:07,  1.17it/s]64
64
Testing:   4%|▍         | 6/154 [00:04<02:01,  1.22it/s]64
64
Testing:   5%|▍         | 7/154 [00:05<02:08,  1.15it/s]64
64
Testing:   5%|▌         | 8/154 [00:07<02:32,  1.05s/it]64
64
Testing:   6%|▌         | 9/154 [00:07<02:16,  1.06it/s]64
64
Testing:   6%|▋         | 10/154 [00:09<02:40,  1.11s/it]64
64
Testing:   7%|▋         | 11/154 [00:10<02:41,  1.13s/it]64
64
Testing:   8%|▊         | 12/154 [00:11<02:19,  1.02it/s]64
64
Testing:   8%|▊         | 13/154 [00:12<02:12,  1.07it/s]64
64
Testing:   9%|▉         | 14/154 [00:12<01:55,  1.21it/s]64
64
Testing:  10%|▉         | 15/154 [00:13<01:48,  1.28it/s]

[{'test_acc': 0.7996742725372314, 'test_loss': 0.5196648240089417}]

In [2]:
import sys
import pickle
print(sys.executable)

import sklearn
sklearn.__version__
import torchtext
from main_recurrent import *
from collections import Counter

/Users/AS/anaconda3/envs/atcs-nli-representations/bin/python


In [42]:
print(text.vocab.stoi["jesus"])

3303


In [55]:
from torchtext import data

In [54]:
data?

In [ ]:
# Copyright (c) 2017-present, Facebook, Inc.
# All rights reserved.
#
# This source code is licensed under the license found in the
# LICENSE file in the root directory of this source tree.
#

from __future__ import absolute_import, division, unicode_literals

import sys
import io
import numpy as np
import logging
import sklearn

print(sklearn.__version__)

# Set PATHs
# path to senteval
PATH_TO_SENTEVAL = '../SentEval/'
# path to the NLP datasets 
# PATH_TO_DATA = '../data/downstream'
PATH_TO_DATA = '../SentEval/data'
# path to glove embeddings
PATH_TO_VEC = '../SentEval/pretrained/glove.840B.300d.txt'


# import SentEval
sys.path.insert(0, PATH_TO_SENTEVAL)
import senteval
from torchtext.data import Field
import data




# def prepare(params, samples):
#     data = 
# #     counter = create_counter(samples)
# #     params.embeddings = torchtext.vocab.Vocab(counter = counter,
# #                                               vectors = torchtext.vocab.GloVe(name='840B'))
#     return


# 
# def prepare(params, samples):
#     counter = create_counter(samples)
#     params.embeddings = torchtext.vocab.Vocab(counter = counter, vectors = torchtext.vocab.GloVe(name='840B'))
#     return

# def batcher(params, batch):
#     batch = [sent if sent != [] else ['.'] for sent in batch]
#     embeddings = []
  
#     embedding_layer = nn.Embedding.from_pretrained(params.embeddings.vectors, freeze= True)                           
#     model.embedding = embedding_layer


def prepare(params, samples):
    params.inputs.build_vocab(samples)
    params.inputs.vocab.load_vectors('glove.840B.300d')
    params.model.nli_net.encoder.embedding.weight.data = params.inputs.vocab.vectors

def batcher(params, batch):

    sentences = []
    sentences_lens = []
    c = 0
    for s in batch:
        sentences.append(s)
        if len(s) == 0:
            print(s)
        s = '.'
        sentences_lens.append(len(s))
    sentences = params.inputs.process(sentences)[0]
    sentences = sentences.T
    sentence_lens = torch.Tensor(sentences_lens)
    emb = params.model.nli_net.encode((sentences, sentence_lens))
    return emb.detach().numpy()


# Set params for SentEval
# we use logistic regression (usepytorch: Fasle) and kfold 10
# In this dictionary you can add extra information that you model needs for initialization
# for example the path to a dictionary of indices, of hyper parameters
# this dictionary is passed to the batched and the prepare fucntions



# params_senteval = {'task_path': PATH_TO_DATA, 'usepytorch': False, 'kfold': 10,
#                   'tenacity': 3}



#Parameters from Matyas
params_senteval = {'task_path': PATH_TO_DATA, 'usepytorch': False, 'kfold': 10}
params_senteval['classifier'] = {'nhid': 0, 'optim': 'adam', 'batch_size': 64,
                                 'tenacity': 5, 'epoch_size': 4}


# 
# params_senteval = {'task_path': PATH_TO_DATA, 'usepytorch': False, 'kfold': 10}


# this is the config for the NN classifier but we are going to use scikit-learn logistic regression with 10 kfold
# usepytorch = False 
#params_senteval['classifier'] = {'nhid': 0, 'optim': 'rmsprop', 'batch_size': 128,
#                                 'tenacity': 3, 'epoch_size': 2}

# Set up logger
logging.basicConfig(format='%(asctime)s : %(message)s', level=logging.DEBUG)

# if __name__ == "__main__":
def seval(model):
    
    
    
    params_senteval['model'] =  model
    params_senteval['inputs'] = Field(lower=True, tokenize=word_tokenize, include_lengths=True)
    
    # here you define the NLP taks that your embedding model is going to be evaluated
    # in (https://arxiv.org/abs/1802.05883) we use the following :
    # SICKRelatedness (Sick-R) needs torch cuda to work (even when using logistic regression), 
    # but STS14 (semantic textual similarity) is a similar type of semantic task
    se = senteval.engine.SE(params_senteval, batcher, prepare)
    transfer_tasks = ['MR','CR', 'SUBJ', 'MPQA',  'SST2', 'SST5', 'TREC', 'MRPC', 'SICKEntailment' ]
#     'STS12', 'STS13', 'STS14', 'STS15', 'STS16'
                        # 'SICKRelatedness', 'STSBenchmark'
#                       'MPQA', 'SUBJ', 'SST2', 'TREC',
#                       'MRPC', 'SICKEntailment']
#                       , 'STS14']
    # senteval prints the results and returns a dictionary with the scores
#     
    results_LSTM = se.eval(transfer_tasks)
    print(results_LSTM)


    save_obj(results_dummy, 'LSTM_output')
    results_recovered = load_obj('LSTM_output')

seval(model_test)

/Users/AS/anaconda3/envs/atcs-nli-representations/lib/python3.9/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
2021-04-18 23:45:21,819 : ***** Transfer task : MR *****


2021-04-18 23:45:21,824 : Loading vectors from .vector_cache/glove.840B.300d.txt.pt


0.24.1


2021-04-18 23:45:30,142 : Generating sentence embeddings
2021-04-18 23:45:30,262 : Generated sentence embeddings
2021-04-18 23:45:30,264 : Training sklearn-LogReg with (inner) 10-fold cross-validation


74


2021-04-18 23:45:31,114 : Best param found at split 1: l2reg = 0.25                 with score 58.33
2021-04-18 23:45:31,835 : Best param found at split 2: l2reg = 8                 with score 63.1
2021-04-18 23:45:32,595 : Best param found at split 3: l2reg = 0.5                 with score 53.33
2021-04-18 23:45:33,322 : Best param found at split 4: l2reg = 0.25                 with score 60.95
2021-04-18 23:45:34,050 : Best param found at split 5: l2reg = 0.25                 with score 53.57
2021-04-18 23:45:34,796 : Best param found at split 6: l2reg = 0.25                 with score 56.43
2021-04-18 23:45:35,542 : Best param found at split 7: l2reg = 1                 with score 55.0
2021-04-18 23:45:36,295 : Best param found at split 8: l2reg = 0.25                 with score 59.76
2021-04-18 23:45:37,007 : Best param found at split 9: l2reg = 0.25                 with score 53.57
2021-04-18 23:45:37,745 : Best param found at split 10: l2reg = 2                 with score 59.76
2

...

/Users/AS/anaconda3/envs/atcs-nli-representations/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/AS/anaconda3/envs/atcs-nli-representations/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-le

/Users/AS/anaconda3/envs/atcs-nli-representations/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/AS/anaconda3/envs/atcs-nli-representations/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-le

/Users/AS/anaconda3/envs/atcs-nli-representations/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/AS/anaconda3/envs/atcs-nli-representations/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-le

/Users/AS/anaconda3/envs/atcs-nli-representations/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/AS/anaconda3/envs/atcs-nli-representations/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-le

/Users/AS/anaconda3/envs/atcs-nli-representations/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
